In [2]:
import pickle
import pandas as pd
import psycopg2 as pg
import geopandas as gpd

from Data_getter.get_data import get_buildings, get_service
from Data_getter.prepare_data_to_balance import DataPreparation
from Blocks_getter.blocks_getter import BlocksModel
from Provision_getter.provision_getter import ProvisionModel
from Balancer.balancer import MasterPlan

with open('output_data/balanced_result_block.pkl', 'rb') as f:
    updated_block = pickle.load(f)

# GROUPS = ['schools', 'kindergartens', 'recreational_areas']

city_crs = 32636
city_id = 1
service_type = "schools"

matrix = pd.read_pickle("/home/gk/jupyter/MMG_SPB.pkl")
engine = pg.connect("dbname='city_db_final' user='postgres' host='10.32.1.107' port='5432' password='postgres'")
buildings_gdf = get_buildings(engine=engine, city_crs=city_crs, city_id=city_id)
service_gdf = get_service(engine=engine, city_crs=city_crs, city_id=city_id, service_type=service_type)
# blocks_gdf = BlocksModel(city_name='Санкт-Петербург', city_admin_level=5, city_crs=city_crs).get_blocks()
# buildings = gpd.read_parquet('/home/gk/jupyter/masterplanning/output_data/buildings.parquet')
blocks_gdf = gpd.read_parquet('/home/gk/jupyter/masterplanning/output_data/blocks.parquet')
services_accessabilities = {"schools": 30, "kindergartens": 30, "recreational_areas": 30}

service_name = "schools"
model = ProvisionModel(
    SUB_GROUP=service_name,
    blocks=blocks_gdf,
    matrix=matrix,
    buildings=buildings_gdf,
    ACCS_TIME=services_accessabilities[service_name],
    service_gdf=service_gdf
)
service_provision_gdf = model.run()
blocks_with_buildings_info = DataPreparation(buildings_gdf, blocks_gdf).run()

/usr/local/lib/python3.8/dist-packages/geopandas/io/sql.py:154: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(
/usr/local/lib/python3.8/dist-packages/geopandas/io/sql.py:154: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(
100%|██████████| 605/605 [00:13<00:00, 45.47it/s]


количество кварталов c сервисом schools: 51
количество жилых кварталов: 2110
количество кварталов всего: 5522
количество кварталов c ошибкой: 0


100%|██████████| 152800/152800 [00:18<00:00, 8046.80it/s] 


In [9]:
Hectare = 10000
gdf = blocks_with_buildings_info.copy()
school = service_provision_gdf.copy()
gdf['current_green_area'] = 0
gdf["current_building_area"] = gdf["current_living_area"] + gdf["current_industrial_area"]
gdf_ = gdf[
    [
        "block_id",
        "area",
        "current_living_area",
        "current_industrial_area",
        "current_population",
        "current_green_area",
        "floors",
    ]
]
gdf_ = gdf_.merge(school[["id", "population_unprov_schools"]], left_on="block_id", right_on="id")
gdf_.drop(['id'], axis=1, inplace=True)

gdf_['area'] = gdf_['area']/Hectare
gdf_['current_living_area'] = gdf_['current_living_area']/Hectare
gdf_['current_industrial_area'] = gdf_['current_industrial_area']/Hectare
gdf_['current_green_area'] = gdf_['current_green_area']/Hectare

# sample = gdf_[gdf_['current_population']!=0].sample()
gdf_ = gdf_[gdf_['area']>10]


In [7]:
gdf_.head()

,block_id,area,current_living_area,current_industrial_area,current_population,current_green_area,floors,population_unprov_schools
0,0,273.897213,0.000000,0.120807,0.0,0.0,0.0,0
1,1,279.537418,1.410618,1.662523,703.0,0.0,1.0,0
3,3,26.805209,0.017208,0.102946,10.0,0.0,1.5,0
4,4,238.060021,0.111216,7.347711,51.0,0.0,1.0,0
5,5,16.600908,0.274299,0.294492,84.0,0.0,1.0,0


In [10]:
from tqdm import tqdm
tqdm.pandas()

invalid_blocks = list()

for block_id in tqdm(gdf_['block_id']):
    try:
        sample = gdf_.loc[block_id, :]
        sample = sample.to_dict('records')
        block = sample[0].copy()
        mp = MasterPlan(area=block['area'], current_living_area=block['current_living_area'],
                        current_industrial_area=block['current_industrial_area'],
                        current_population=block['current_population'],
                        current_green_area=block['current_green_area'],
                        current_unprov_schoolkids=block['population_unprov_schools'],
                        current_unprov_kids=block['population_unprov_kindergartens'],
                        current_unprov_green_population=block['population_unprov_recreational_areas'])
        mp.func()
        res = mp.optimal_values(*mp.select_optimal())
        res['block_id'] = block['block_id']
    except Exception:
        invalid_blocks.append(block_id)

100%|██████████| 2006/2006 [00:00<00:00, 4698.35it/s]


In [12]:
len(invalid_blocks)

2006